In [89]:
import pandas as pd
import numpy as np
import re # FOR REGEX

In [90]:
# download data
url = 'https://www150.statcan.gc.ca/n1/pub/61-517-x/2019004/ICOCD_ICOE.CSV'
raw_data = pd.read_csv(url, delimiter = ';')
raw_data.to_csv('../data/raw/cov_business_license.csv', index=False)

In [225]:
# read in data
cov_data = pd.read_csv('../data/raw/cov_business_license.csv', dtype={20:'str'})
stat_canada_data = pd.read_csv('../data/raw/Stat_Can_inter_corp.csv')

/opt/miniconda3/envs/edsc/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [226]:
# clean up
stat_canada_data.columns=['CCID','LINKAGE','PID','NAICS','CCTL','CRES','PRES','TYPE','NAME','COP','RECTYPE','COMMENT','LEVEL','RQSTID','RECID','BN9']
stat_canada_data.head(3)

,CCID,LINKAGE,PID,NAICS,CCTL,CRES,PRES,TYPE,NAME,COP,RECTYPE,COMMENT,LEVEL,RQSTID,RECID,BN9
0,89354401,Y,89411003,22,CAN,CAN,AB,CC,ENMAX ENERGY CORPORATION ...,100.0,C,...,3,3278,41084,0
1,89354641,Y,89354641,55,CAN,CAN,AB,CC,400369 ALBERTA LTD ...,0.0,C,...,1,625,7394,0
2,89354641,Y,89354641,55,CAN,CAN,AB,CC,400369 ALBERTA LTD ...,0.0,E,...,0,625,7393,0


In [227]:
# fix names for matching
cov_data = cov_data.assign(BusinessName_join = cov_data.BusinessName.str.lower())
# cov_data['BusinessName_join'] = cov_data['BusinessName_join'].str.replace(r'\sltd[\s$]| \slimited[\s$]','', regex=True)
# cov_data['BusinessName_join'] = cov_data['BusinessName_join'].str.replace(r'\sinc[\s$]| \sincorporated[\s$]','', regex=True)
# cov_data['BusinessName_join'] = cov_data['BusinessName_join'].str.replace(r'\scorp[\s$]| \scorporation[\s$]','', regex=True)

In [228]:
cov_data = cov_data.assign(BusinessTradeName_join = cov_data.BusinessTradeName.str.lower())
cov_data = cov_data.assign(BusinessName_join = cov_data.BusinessName_join.str.replace(r"[^\w]", "", regex=True))
cov_data = cov_data.assign(BusinessTradeName_join = cov_data.BusinessTradeName_join.str.replace(r"[^\w]", "", regex=True))
cov_data.head(2)

,FOLDERYEAR,LicenceRSN,LicenceNumber,LicenceRevisionNumber,BusinessName,BusinessTradeName,Status,IssuedDate,ExpiredDate,BusinessType,...,Province,Country,PostalCode,LocalArea,NumberofEmployees,FeePaid,ExtractDate,Geom,BusinessName_join,BusinessTradeName_join
0,14,2180123,14-160030,0,Golden Trim Enterprises Inc,Floorhaus,Issued,2013-12-05T16:08:53-08:00,2014-12-31,Wholesale Dealer w/ Anc. Retail,...,BC,CA,V5Y 1G1,Mount Pleasant,4.0,262.0,2019-07-21T13:49:13-07:00,"{""type"": ""Point"", ""coordinates"": [-123.1098925...",goldentrimenterprisesinc,floorhaus
1,14,2180373,14-160220,0,Tamton Networking Inc,Tam Ton Training,Issued,2014-02-26T13:08:59-08:00,2014-12-31,Instruction,...,BC,CA,V6B 6N9,Downtown,0.0,130.0,2019-07-21T13:49:13-07:00,"{""type"": ""Point"", ""coordinates"": [-123.1067518...",tamtonnetworkinginc,tamtontraining


In [229]:
# fix names for matching
stat_canada_data['NAME_JOIN'] = stat_canada_data['NAME'].str.lower()
# stat_canada_data['NAME'] = stat_canada_data['NAME'].str.replace(r'\sltd[\s$]| \slimited[\s$]','', regex=True)
# stat_canada_data['NAME'] = stat_canada_data['NAME'].str.replace(r'\sinc[\s$]| \sincorporated[\s$]','', regex=True)
# stat_canada_data['NAME'] = stat_canada_data['NAME'].str.replace(r'\scorp[\s$]| \scorporation[\s$]','', regex=True)
# remove spaces & punctuation
stat_canada_data['NAME_JOIN'] = stat_canada_data['NAME_JOIN'].str.replace(r'[^\w]', '', regex=True)

In [230]:
# create rankings
stat_canada_data_ranked = stat_canada_data
stat_canada_data_ranked = stat_canada_data_ranked.assign(ranking = stat_canada_data_ranked.groupby("NAME_JOIN")["COP"].rank(ascending=False).astype("int"))
stat_canada_data_ranked = stat_canada_data_ranked.query("ranking ==1")
stat_canada_data_ranked.head(2)

,CCID,LINKAGE,PID,NAICS,CCTL,CRES,PRES,TYPE,NAME,COP,RECTYPE,COMMENT,LEVEL,RQSTID,RECID,BN9,NAME_JOIN,ranking
0,89354401,Y,89411003,22,CAN,CAN,AB,CC,ENMAX ENERGY CORPORATION ...,100.0,C,...,3,3278,41084,0,enmaxenergycorporation,1
1,89354641,Y,89354641,55,CAN,CAN,AB,CC,400369 ALBERTA LTD ...,0.0,C,...,1,625,7394,0,400369albertaltd,1


In [233]:
# merge both dataframes
merged_df = cov_data.merge(stat_canada_data_ranked, how='inner', left_on='BusinessName_join', right_on='NAME_JOIN')

# try remaining unmatched on second name column
cov_data_remainder = cov_data.merge(stat_canada_data_ranked, how='outer', left_on='BusinessName_join', right_on='NAME_JOIN').iloc[:,:26]
merged_df_2 = cov_data_remainder.merge(stat_canada_data_ranked, how='inner', left_on='BusinessTradeName_join', right_on='NAME_JOIN')
merged_df = pd.concat([merged_df,merged_df_2], axis=0)

# save processed dataframe
merged_df.to_csv('../data/processed/combined_data.csv', index=False)

In [583]:
# compile list of company names & save as csv
name_list = merged_df[['BusinessName','BusinessTradeName','NAME']]

name_list = name_list.melt(var_name='source', value_name = 'co_name')['co_name']
name_list = pd.DataFrame({'co_name': name_list.unique()})
name_list.to_csv('data/processed/name_list.csv', index=False)

## Investment Discipline

Field names:

- CTL: Indicates the **country of control.**
- NAICS: Indicates the North American Industrial Classification System code. **(industry)**
- PCT: Indicates the **percentage of voting rights** owned, usually in the current corporation by some preceding corporation (see Level); except, however, for "INV" corporations (see below), which hold the indicated portion of the enterprise parent.
- PRV: Indicates the province or territory, and in some cases the **state of residence of the corporation.**
- RES: Indicates the **country of residence.**

Comment:

A comment is a note indicating a distinct characteristic or ownership situation for a corporation. When applicable, this information will be found in the detail pane at the bottom of the page, and will also be noticeable by a blue lettered icon on the right hand side the corporation in the structure. Here a list of the various comments and their meaning:

- A: Indicates that control has been assigned for reasons other than ownership of voting rights.
- D: Indicates the corporation is not controlled by this enterprise.
- R: Indicates that the corporation has already been listed within this enterprise, but it has more than one holding company. The companies that are in turn held are not shown again.
- AD: Indicates that control has been assigned to a different enterprise.
- AR: Indicates that control has been assigned to this enterprise, and it is a repeated listing.
- SOE: Indicates the corporation is a Foreign State-Owned-Enterprise.
- SIE: Indicates the corporation is a Foreign State-Influenced-Enterprise.
- A-SOE: Indicates that control has been assigned for reasons other than ownership of voting rights and that this corporation is a Foreign State-Owned-Enterprise.
- D-SIE: Indicates that the corporation is not controlled by this enterprise but that it is a Foreign State-Influenced-Enterprise.
- R-SOE: Indicates that the Foreign State-Owned-Enterprise has already been listed within this enterprise, but it has more than one holding company. The companies that are in turn held are not shown again.

About multiple entries

**Multiple entries for the same corporation**, in the corporation list search results, represents the **presence of more than one ownership link** for this corporation. As such, a corporation with **two or more owners will be displayed two or more times** accordingly. Please note that if the first entry in the list is selected, the application will bring the user to the first instance the corporation is found in the database. If the second entry is selected, then it will bring the user to the second instance the corporation is found, and so on.

Levels of Ownership

Enterprise parents are shown as level "1", their direct subsidiaries as level "2", and indirectly held corporations are attributed higher level numbers. As such, a level "3" corporation is held directly by the first level "2" listed above it.